In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=2048,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [3]:
client = InferenceClient(
    ""
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [12]:
!mkdir mixtral-factual-wrong-v3
# !rm mixtral-factual-wrong-v2/*.json

In [13]:
# prompt = 'generate more factual wrong questions ONLY, like, why kangaroo in malaysia, can expand beyond animals'
# formatted_prompt = format_prompt(prompt, [])
# stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
# output = stream.generated_text
# output

In [14]:
# prompt = 'generate more factual wrong and stupid malaysian context questions ONLY, like, kenapa kerajaan malaysia bodoh sangat'
# formatted_prompt = format_prompt(prompt, [])
# stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
# output = stream.generated_text
# output

In [15]:
def answer(q, i):
    filename = f'mixtral-factual-wrong-v3/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [16]:
prompts = ['generate more factual wrong questions ONLY, like, why kangaroo in malaysia, can expand beyond animals'] * 5000

In [17]:
len(prompts)

5000

In [18]:
max_worker = 50

questions = prompts
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|█████████████████████████████████████████| 100/100 [41:51<00:00, 25.11s/it]


In [175]:
from glob import glob

files = glob('mixtral-factual-wrong-v3/*.json')
files = sorted(files, key = lambda x: int(x.split('/')[-1].replace('.json', '')))
        
len(files)

5000

In [176]:
import json

questions = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
    splitted = data.strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    splitted = [s[:-1] if s[-1] == '"' else s for s in splitted]
    splitted = [s.strip() for s in splitted]
    questions.extend(splitted)
    
questions = sorted(list(set(questions)))
len(questions)

39164

In [177]:
questions[:10]

['Can I buy a pet unicorn in Canada?',
 'Can I find a kangaroo in England?',
 'Can I find a penguin in the African desert?',
 'Can I find a snowy owl in the Amazon rainforest?',
 'Can I go on a gorilla trek in the Arctic?',
 'Can I go on a polar bear trek in the African savannah?',
 'Can I go whale watching in the Great Lakes?',
 'Can I grow bananas in the Artic tundra?',
 'Can I keep a lion as a pet in France?',
 'Can I see flamingos in the Siberian tundra?']

In [22]:
!mkdir answer-mixtral-factual-wrong-v3

In [23]:
def answer(q, i):
    filename = f'answer-mixtral-factual-wrong-v3/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [25]:
max_worker = 50

for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████| 784/784 [4:03:34<00:00, 18.64s/it]


In [26]:
data = []
for f in tqdm(glob('answer-mixtral-factual-wrong-v3/*.json')):
    with open(f) as fopen:
        data.append(json.load(fopen).strip())
        
data.extend(questions)
len(data)

100%|█████████████████████████████████| 39164/39164 [00:00<00:00, 110696.56it/s]


78328

In [27]:
with open('answer-mixtral-factual-wrong-v3.texts', 'w') as fopen:
    for t in set(data):
        fopen.write(f'{json.dumps(t)}\n')

In [28]:
!cp answer-mixtral-factual-wrong-v3.texts ../translation

In [171]:
import re

mapping = {}
for f in glob('/home/husein/ssd3/translation/answer-mixtral-factual-wrong-v3.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
len(mapping)

78324

In [172]:
# 'cannot provide assistance',
# 'disseminating sensitive government information',
# 'strongly advise against any illegal',
# 'cannot assist with that',
# 'cannot condone or assist',
# 'cannot assist you',
# 'cannot provide guidance',
# 'committed to promoting safety',
# 'purpose is to provide helpful',
# 'strongly advise against engaging',
# 'AI language model'

In [178]:
with open('mixtral-factual-wrong-v2.jsonl', 'w') as fopen_l:
    for no, q in enumerate(questions):
        filename = f'answer-mixtral-factual-wrong-v3/{no}.json'
        if not os.path.exists(filename):
            continue

        with open(filename) as fopen:
            a = json.load(fopen).strip()
            
        if a.count('?') > 0:
            continue
            
        d = {
            'question': q,
            'question_ms': mapping.get(q),
            'answer': a,
            'answer_ms': mapping.get(a)
        }
        fopen_l.write(f'{json.dumps(d)}\n')

In [179]:
!head -n 3 mixtral-factual-wrong-v2.jsonl

{"question": "Can I buy a pet unicorn in Canada?", "question_ms": "Bolehkah saya membeli unicorn haiwan peliharaan di Kanada?", "answer": "I wish it were possible to buy a real-life pet unicorn, but unfortunately, they are mythical creatures and do not exist. You can find many items related to unicorns, like toys, decorations, and costumes, in Canada and other countries, but a real, living unicorn is not available for purchase anywhere in the world.", "answer_ms": "Saya berharap mungkin untuk membeli unicorn haiwan peliharaan sebenar, tetapi malangnya, ia adalah makhluk mitos dan tidak wujud. Anda boleh menemui banyak item yang berkaitan dengan unicorn, seperti mainan, hiasan dan pakaian, di Kanada dan negara lain, tetapi unicorn sebenar yang hidup tidak tersedia untuk pembelian di mana-mana di dunia."}
{"question": "Can I find a kangaroo in England?", "question_ms": "Bolehkah saya mencari kanggaru di England?", "answer": "No, you would not normally find a kangaroo in England. Kangaroo

In [180]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mixtral-factual-wrong-v2.jsonl',
    path_in_repo='mixtral-factual-wrong-v2.jsonl',
    repo_id='mesolitica/mixtral-malaysian-general-qa',
    repo_type='dataset',
)

mixtral-factual-wrong-v2.jsonl:   0%|          | 0.00/56.7M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/mixtral-malaysian-general-qa/blob/main/mixtral-factual-wrong-v2.jsonl'